# Notebook Summary

In this notebook I optimize a KNN model

In [1]:
import autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import Classes

In [2]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import ADASYN
from sklearn.neighbors import KNeighborsClassifier

# Pickle in Data

In [3]:
# Pickle in cleaned dataframe

# Designate path

path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\df_agg.pkl"

df = pickle.load(open(path,'rb'))
df.head(2)

,level_0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,TikTok Link,Release Date,Position Change,spotify_uri,audio_analysis,feature_analysis,success,year,top_albums,top_artists
0,0,0.880,0.501,2.0,-6.774,1.0,0.062,0.0494,0.0695,0.4360,...,https://www.tiktok.com/music/All-TikTok-Mashup...,2020-08-17,23.0,5TpvLkESnw1g9wDz52efeO,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.88, 'energy': 0.501, 'key':...",1,2020.0,Other,Other
1,162,0.935,0.454,1.0,-7.509,1.0,0.375,0.0194,0.0000,0.0824,...,https://www.tiktok.com/music/WAP-Megan-Thee-St...,2018-03-22,15.0,4Oun2ylbjFKMPTiaSbbCih,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.935, 'energy': 0.454, 'key'...",1,2018.0,Other,Cardi B


# CV KNN

In [18]:
# Seperate features from label

X = df.loc[:,['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature','Album Name','Artist(s)', 'year']]

y = df['success']

In [19]:
# Get dummy varialbes
Classes.gnumeric_func(X, ['Artist(s)','Album Name'])

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,Album Name,Artist(s),year
0,0.880,0.5010,2.0,-6.774,1.0,0.0620,0.049400,0.069500,0.4360,0.4590,120.038,4.0,0,0,2020.0
1,0.935,0.4540,1.0,-7.509,1.0,0.3750,0.019400,0.000000,0.0824,0.3570,133.073,4.0,1,1,2018.0
2,0.842,0.5970,2.0,-6.336,1.0,0.0627,0.002520,0.000000,0.1240,0.2980,145.992,4.0,2,2,2018.0
3,0.884,0.5460,2.0,-6.279,0.0,0.1170,0.269000,0.000008,0.0640,0.4900,113.236,3.0,3,3,2020.0
4,0.926,0.7620,1.0,-1.887,1.0,0.2050,0.000718,0.000000,0.1170,0.2770,127.931,4.0,4,4,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670,0.626,0.4150,2.0,-12.073,1.0,0.6030,0.559000,0.000000,0.0931,0.5940,99.609,4.0,7338,1259,0.0
7671,0.672,0.0304,2.0,-26.380,1.0,0.0381,0.288000,0.865000,0.0676,0.0682,90.037,4.0,7339,2637,0.0
7672,0.879,0.6420,0.0,-6.775,1.0,0.0570,0.096800,0.000002,0.1300,0.7420,129.938,4.0,7340,5394,0.0
7673,0.649,0.9310,1.0,-3.150,0.0,0.1810,0.009300,0.000000,0.7510,0.7440,153.645,4.0,7341,20,0.0


In [20]:
#Split data into 3: 60% train, 20% validation, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [21]:
# Oversample

ada = ADASYN(random_state=42)

# Oversample training data
X_adasyn_tr, y_adasyn_tr = ada.fit_resample(X_train,y_train)

In [22]:
# Scale features 
scaler = StandardScaler()
X_adasyn_tr = scaler.fit_transform(X_adasyn_tr)

In [23]:
# specify "parameter distributions" rather than a "parameter grid"
k_range = list(range(1, 31))
weight_options = ['uniform', 'distance']
param_dist = dict(n_neighbors=k_range, weights=weight_options)

In [24]:
# create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range, weights=weight_options)
print(param_grid)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], 'weights': ['uniform', 'distance']}


In [25]:
# Define KNN
knn = KNeighborsClassifier()

In [26]:
# n_iter controls the number of searches
rand = RandomizedSearchCV(knn, param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=42)
rand.fit(X_adasyn_tr, y_adasyn_tr)
rand.cv_results_

{'mean_fit_time': array([0.08114779, 0.08097432, 0.07934027, 0.08644981, 0.09458275,
        0.08248725, 0.08355494, 0.07797544, 0.07810593, 0.07718754]),
 'std_fit_time': array([0.003374  , 0.00596111, 0.00442849, 0.01003317, 0.01272706,
        0.00579878, 0.00690861, 0.00306035, 0.00324899, 0.00347152]),
 'mean_score_time': array([0.16210167, 0.17415512, 0.25064533, 0.25578697, 0.23216016,
        0.26734626, 0.22258525, 0.25059602, 0.20297759, 0.22689936]),
 'std_score_time': array([0.01703159, 0.00980666, 0.00872997, 0.01694101, 0.02714172,
        0.01299541, 0.00732917, 0.00737522, 0.01133376, 0.00754966]),
 'param_weights': masked_array(data=['uniform', 'distance', 'uniform', 'distance',
                    'distance', 'uniform', 'distance', 'uniform',
                    'uniform', 'distance'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_n_neighbors': 